# Trying to vectorise the prediction logic

## Setup the data

In [16]:
import pandas as pd
import json

In [17]:
# Load data
with open('estimated_travel_times.json', 'r') as f:
    estimated_travel_times = json.load(f)
journeys = pd.read_csv("journeys.csv", parse_dates=["from_date", "to_date"])
journeys.head()

,plate,from_date,to_date,from_location,to_location
0,ABC123,2024-01-15 08:30:00,2024-01-15 10:45:00,Auckland,Hamilton
1,ABC123,2024-01-15 09:15:00,2024-01-15 11:20:00,Wellington,Palmerston North
2,ABC123,2024-01-15 07:45:00,2024-01-15 09:30:00,Christchurch,Timaru
3,ABC123,2024-01-15 13:20:00,2024-01-15 15:45:00,Hamilton,Tauranga
4,ABC123,2024-01-15 14:10:00,2024-01-15 16:25:00,Dunedin,Invercargill


## Do prediction

In [18]:
# Calculate travel times
journeys["actual_travel_time"] = journeys["to_date"] - journeys["from_date"]
journeys["actual_travel_time"] = journeys["actual_travel_time"].dt.total_seconds().astype(int) // 60
journeys["estimated_travel_time"] = journeys.apply(
    lambda journey: 
        estimated_travel_times.get(journey["from_location"], {}).get(
            journey["to_location"], None
    ),
    axis=1,
)

journeys.head()

,plate,from_date,to_date,from_location,to_location,actual_travel_time,estimated_travel_time
0,ABC123,2024-01-15 08:30:00,2024-01-15 10:45:00,Auckland,Hamilton,135,135.0
1,ABC123,2024-01-15 09:15:00,2024-01-15 11:20:00,Wellington,Palmerston North,125,145.0
2,ABC123,2024-01-15 07:45:00,2024-01-15 09:30:00,Christchurch,Timaru,105,165.0
3,ABC123,2024-01-15 13:20:00,2024-01-15 15:45:00,Hamilton,Tauranga,145,75.0
4,ABC123,2024-01-15 14:10:00,2024-01-15 16:25:00,Dunedin,Invercargill,135,210.0


In [19]:
# Calculate break times
journeys["break_time"] = journeys["actual_travel_time"] - journeys["estimated_travel_time"]
journeys.head()

,plate,from_date,to_date,from_location,to_location,actual_travel_time,estimated_travel_time,break_time
0,ABC123,2024-01-15 08:30:00,2024-01-15 10:45:00,Auckland,Hamilton,135,135.0,0.0
1,ABC123,2024-01-15 09:15:00,2024-01-15 11:20:00,Wellington,Palmerston North,125,145.0,-20.0
2,ABC123,2024-01-15 07:45:00,2024-01-15 09:30:00,Christchurch,Timaru,105,165.0,-60.0
3,ABC123,2024-01-15 13:20:00,2024-01-15 15:45:00,Hamilton,Tauranga,145,75.0,70.0
4,ABC123,2024-01-15 14:10:00,2024-01-15 16:25:00,Dunedin,Invercargill,135,210.0,-75.0


In [20]:
# determine if a journey includes a break
break_length = 30
journeys["is_break"] = journeys["break_time"] >= break_length
journeys.head()

,plate,from_date,to_date,from_location,to_location,actual_travel_time,estimated_travel_time,break_time,is_break
0,ABC123,2024-01-15 08:30:00,2024-01-15 10:45:00,Auckland,Hamilton,135,135.0,0.0,False
1,ABC123,2024-01-15 09:15:00,2024-01-15 11:20:00,Wellington,Palmerston North,125,145.0,-20.0,False
2,ABC123,2024-01-15 07:45:00,2024-01-15 09:30:00,Christchurch,Timaru,105,165.0,-60.0,False
3,ABC123,2024-01-15 13:20:00,2024-01-15 15:45:00,Hamilton,Tauranga,145,75.0,70.0,True
4,ABC123,2024-01-15 14:10:00,2024-01-15 16:25:00,Dunedin,Invercargill,135,210.0,-75.0,False


In [21]:
# group continuous work time sessions
journeys = journeys.sort_values(by="from_date")
journeys["work_session_number"] = journeys["is_break"].cumsum().shift(fill_value=0)
journeys.head()

,plate,from_date,to_date,from_location,to_location,actual_travel_time,estimated_travel_time,break_time,is_break,work_session_number
368,ABC123,2024-01-15 00:00:28,2024-01-17 18:17:01,Napier,Cambridge,3976,305.0,3671.0,True,0
625,ABC123,2024-01-15 00:06:45,2024-01-16 03:36:40,Invercargill,Auckland,1649,900.0,749.0,True,1
525,ABC123,2024-01-15 00:16:11,2024-01-17 05:53:44,Taupo,Hamilton,3217,150.0,3067.0,True,2
119,ABC123,2024-01-15 00:24:29,2024-01-15 09:23:35,Hamilton,Wellington,539,325.0,214.0,True,3
129,ABC123,2024-01-15 00:29:37,2024-01-15 13:25:45,Hamilton,Dunedin,776,585.0,191.0,True,4


In [22]:
# calculate cumulative work time for each session
journeys["work_session_cumulative_work_time"] = journeys.groupby(by="work_session_number")['actual_travel_time'].cumsum()
journeys.head()

,plate,from_date,to_date,from_location,to_location,actual_travel_time,estimated_travel_time,break_time,is_break,work_session_number,work_session_cumulative_work_time
368,ABC123,2024-01-15 00:00:28,2024-01-17 18:17:01,Napier,Cambridge,3976,305.0,3671.0,True,0,3976
625,ABC123,2024-01-15 00:06:45,2024-01-16 03:36:40,Invercargill,Auckland,1649,900.0,749.0,True,1,1649
525,ABC123,2024-01-15 00:16:11,2024-01-17 05:53:44,Taupo,Hamilton,3217,150.0,3067.0,True,2,3217
119,ABC123,2024-01-15 00:24:29,2024-01-15 09:23:35,Hamilton,Wellington,539,325.0,214.0,True,3,539
129,ABC123,2024-01-15 00:29:37,2024-01-15 13:25:45,Hamilton,Dunedin,776,585.0,191.0,True,4,776


In [23]:
# Assess breaches
breach_threshold = 500
journeys["breach"] = journeys["work_session_cumulative_work_time"] > breach_threshold
journeys["breach_amount"] = journeys["work_session_cumulative_work_time"] - breach_threshold
journeys.head()


,plate,from_date,to_date,from_location,to_location,actual_travel_time,estimated_travel_time,break_time,is_break,work_session_number,work_session_cumulative_work_time,breach,breach_amount
368,ABC123,2024-01-15 00:00:28,2024-01-17 18:17:01,Napier,Cambridge,3976,305.0,3671.0,True,0,3976,True,3476
625,ABC123,2024-01-15 00:06:45,2024-01-16 03:36:40,Invercargill,Auckland,1649,900.0,749.0,True,1,1649,True,1149
525,ABC123,2024-01-15 00:16:11,2024-01-17 05:53:44,Taupo,Hamilton,3217,150.0,3067.0,True,2,3217,True,2717
119,ABC123,2024-01-15 00:24:29,2024-01-15 09:23:35,Hamilton,Wellington,539,325.0,214.0,True,3,539,True,39
129,ABC123,2024-01-15 00:29:37,2024-01-15 13:25:45,Hamilton,Dunedin,776,585.0,191.0,True,4,776,True,276


## Is it quick?

In [ ]:
import time

In [29]:
# Load data
with open('estimated_travel_times.json', 'r') as f:
    estimated_travel_times = json.load(f)
journeys = pd.read_csv("journeys_long.csv", parse_dates=["from_date", "to_date"])
journeys.head()

print("Loaded journeys data with shape:", journeys.shape)
start_time = time.time()

# Calculate travel times
journeys["actual_travel_time"] = journeys["to_date"] - journeys["from_date"]
journeys["actual_travel_time"] = journeys["actual_travel_time"].dt.total_seconds().astype(int) // 60
journeys["estimated_travel_time"] = journeys.apply(
    lambda journey: 
        estimated_travel_times.get(journey["from_location"], {}).get(
            journey["to_location"], None
    ),
    axis=1,
)

journeys.head()

# Calculate break times
journeys["break_time"] = journeys["actual_travel_time"] - journeys["estimated_travel_time"]


# determine if a journey includes a break
break_length = 30
journeys["is_break"] = journeys["break_time"] >= break_length

# group continuous work time sessions
journeys = journeys.sort_values(by="from_date")
journeys["work_session_number"] = journeys["is_break"].cumsum().shift(fill_value=0)

# calculate cumulative work time for each session
journeys["work_session_cumulative_work_time"] = journeys.groupby(by="work_session_number")['actual_travel_time'].cumsum()


# Assess breaches
breach_threshold = 500
journeys["breach"] = journeys["work_session_cumulative_work_time"] > breach_threshold
journeys["breach_amount"] = journeys["work_session_cumulative_work_time"] - breach_threshold

print("Processed journeys data with shape:", journeys.shape)
end_time = time.time()
print(f"Processing time: {end_time - start_time:.2f} seconds")


Loaded journeys data with shape: (100000, 5)
Processed journeys data with shape: (100000, 13)
Processing time: 0.41 seconds
